# Regresia logistică - proiect de consultanță

## Soluție

## Renunțarea clienților la serviciile unei companii ("customer churn")

O agenție de marketing are mai mulți clienți ce îi folosesc serviciile de producție de publicitate pentru website-uri. Agenția a observat că există un număr de clienți care au încetat sa îi mai folosească serviciile. Agenția atribuie aleator clienților câte un account manager, dar dorește crearea unui model ML pentru predicția clienților ce vor renunța la serviciile sale, astfel încât acestora să le atribuie corect un anumit account manager. 

Pe baza datelor istorice, să se creeze un algoritm de clasificare ce determină dacă un client a renunțat sau nu. Apoi, compania va testa modelul pe date de intrare viitoare, pentru noi clienți, pentru a putea obține predicția celor ce vor renunța, astfel încât să le atribuie acestora un anumit account manager.

Datele se află în fișierul customer_churn.csv, ce conține următoarele coloane:

    Name : Numele ultimului contact in companie
    Age: Varsta clientului
    Total_Purchase: Totalul publicității cumpărate 
    Account_Manager: Binary 0=Niciun manager, 1= Account manager asignat
    Years: Numărul total de ani, în calitate de client
    Num_sites: Numărul de website-uri ce utilizează serviciul
    Onboard_date: Data la care a intrat in companie ultimul contact
    Location: Adresa profesională a clientului
    Company: Numele companiei clientului
    
Odată ce modelul va fi creat și evaluat, acesta va fi testat pe date noi, pe care agenția le-a furnizat în fișierul new_customers.csv (similar unui set reținut din datele primite). Agenția dorește să afle ce clienți vor renunța, cel mai probabil, la serviciile sale, pe baza acestor date (ce încă nu sunt etichetate).

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [ ]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [ ]:
data.printSchema()

### Observarea datelor

In [ ]:
data.describe().show()

In [ ]:
data.columns

### Formatare pentru MLlib

Vom utiliza coloanele numerice și vom include Account Manager fiindcă este simplu, deși probabil nu va indica nimic, știind că agenția a menționat că este atribuit aleator.

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [ ]:
output = assembler.transform(data)

In [ ]:
final_data = output.select('features','churn')

### Divizare Test-Train

In [ ]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Antrenarea modelului

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr_churn = LogisticRegression(labelCol='churn')

In [ ]:
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary

In [ ]:
training_sum.predictions.describe().show()

### Evaluarea rezultatelor

Evaluăm rezultatul pe setul de date primit, folosind datele de test.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [ ]:
pred_and_labels.predictions.show()

### Folosim AUC (Area under curve)

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [ ]:
auc

[Care este valoarea bună pentru AUC?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predicție pe date noi, neetichetate

Evaluarea datelor din fișierul new_customers.csv

In [ ]:
final_lr_model = lr_churn.fit(final_data)

In [ ]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [ ]:
new_customers.printSchema()

In [ ]:
test_new_customers = assembler.transform(new_customers)

In [ ]:
test_new_customers.printSchema()

In [ ]:
final_results = final_lr_model.transform(test_new_customers)

In [ ]:
final_results.select('Company','prediction').show()

Concluzia: Trebuie asignați account managers clienților Cannon-Benson,Barron-Robertson,Sexton-GOlden și  Parks-Robbins.